# Textblob

In [ ]:
# Add the parent directory (news-agg) to sys.path
import sys
from pathlib import Path

parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.append(str(parent_dir))

# Verify the parent directory is added
print("Updated sys.path:")
print(sys.path)


In [ ]:
from sqlmodel import Session, select, create_engine
from app.models import TopHeadline  # Replace with your actual model
from textblob import TextBlob

# Connect to your SQLite database
engine = create_engine("sqlite:///newslist_notebook.db")  # Replace with your database path

# Define a query to fetch rows with a NULL sentiment column
sql_stmt = select(TopHeadline).where(TopHeadline.sentiment.is_(None))

# Start a session to test sentiment analysis
with Session(engine) as session:
    # Fetch rows from the database
    rows = session.exec(sql_stmt).all()
    
    # Loop through each row and analyze the sentiment
    for row in rows:
        analysis = TextBlob(row.content)  # Perform sentiment analysis on the 'content' column
        polarity = analysis.sentiment.polarity  # Get polarity score

        # Determine sentiment
        if polarity < 0:
            row.sentiment = "negative"
        elif polarity == 0:
            row.sentiment = "neutral"
        else:
            row.sentiment = "positive"

        # Print for verification (no commit yet)
        print(f"ID: {row.id}, Sentiment: {row.sentiment}")
    
    # No commit here, changes will not persist
    print("Test completed. Changes are not committed to the database.")
